In [5]:
import pygame
import numpy as np
from typing import List, Tuple
from IPython.display import display, clear_output
import PIL.Image
import time

# Initialize Pygame
pygame.init()

class MovingObject:
    def __init__(self, x: float, y: float, speed: float, target: Tuple[float, float]):
        self.x = x
        self.y = y
        self.speed = speed
        self.target = target
        self.reached = False
        self.color = (np.random.randint(50, 255), np.random.randint(50, 255), np.random.randint(50, 255))

    def move(self):
        if self.reached:
            return

        dx = self.target[0] - self.x
        dy = self.target[1] - self.y
        distance = np.sqrt(dx**2 + dy**2)

        if distance < self.speed:
            self.x = self.target[0]
            self.y = self.target[1]
            self.reached = True
        else:
            self.x += (dx/distance) * self.speed
            self.y += (dy/distance) * self.speed

class TrafficSimulation:
    def __init__(self, width: int = 800, height: int = 600):
        self.width = width
        self.height = height
        self.screen = pygame.Surface((width, height))

        self.nodes = {
            "orig1": (100, 100),
            "orig2": (100, 500),
            "merge": (400, 300),
            "dest": (700, 300)
        }

        self.objects: List[MovingObject] = []
        self.clock = pygame.time.Clock()

    def add_object(self, start: str, end: str, speed: float):
        start_pos = self.nodes[start]
        end_pos = self.nodes[end]
        self.objects.append(MovingObject(start_pos[0], start_pos[1], speed, end_pos))

    def run(self, num_frames=500):
        for frame in range(num_frames):
            # Spawn new objects periodically
            if frame % 120 == 0:  # Spawn every 120 frames
                start_node = "orig1" if np.random.random() < 0.5 else "orig2"
                self.add_object(start_node, "merge", 2)

            # Update object positions
            for obj in self.objects:
                if not obj.reached:
                    obj.move()
                elif obj.target == self.nodes["merge"]:
                    obj.target = self.nodes["dest"]
                    obj.reached = False

            # Remove objects that have reached final destination
            self.objects = [obj for obj in self.objects if not (obj.reached and obj.target == self.nodes["dest"])]

            # Draw
            self.screen.fill((255, 255, 255))

            # Draw nodes
            for pos in self.nodes.values():
                pygame.draw.circle(self.screen, (0, 0, 0), pos, 10)

            # Draw links
            pygame.draw.line(self.screen, (0, 0, 0), self.nodes["orig1"], self.nodes["merge"], 2)
            pygame.draw.line(self.screen, (0, 0, 0), self.nodes["orig2"], self.nodes["merge"], 2)
            pygame.draw.line(self.screen, (0, 0, 0), self.nodes["merge"], self.nodes["dest"], 2)

            # Draw objects
            for obj in self.objects:
                pygame.draw.circle(self.screen, obj.color, (int(obj.x), int(obj.y)), 5)

            # Update display
            pygame_surface = pygame.surfarray.array3d(self.screen)
            pil_image = PIL.Image.fromarray(pygame_surface).transpose(PIL.Image.ROTATE_90)
            clear_output(wait=True)
            display(pil_image)
            
            self.clock.tick(30)  # 30 FPS
            time.sleep(0.03)  # Add a small delay to control the animation speed

# Run the simulation
sim = TrafficSimulation()
sim.run()


Image(value=b'')